In [1]:
epochs = 50

# Parte 7 - Aprendizado federado com Conjunto de Dados Federado

Aqui introduzimos uma nova ferramenta para o uso de Conjunto de Dados (datasets) Federados. Nós criamos a classe `FederatedDataset` cuja intenção é ser usada como a classe Dataset do Pytorch, que é repassado para um carregador de dados federado `FederatedDataLoader` no qual irá iterar sobre os dados de maneira federada.

Autores:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

Tradutor:
- João Lucas - GitHub: [@joaolcaas](https://github.com/joaolcaas)


Usaremos o sandbox que foi discutido no último tutorial

In [ ]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Procure pelo dataset

In [ ]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

Carrega-se um modelo e um otimizador

In [ ]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)

Aqui carregaremos os dados buscados em um `FederatedDataset`. Note que os workers contém parte dos dados

In [ ]:
# Carrega o resultado em BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Construa o objeto FederatedDataset
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

Colocamos os dados em um `FederatedDataLoader` juntamente com informações específicas.

In [ ]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

E finalmente nós iteramos sobre as épocas. É visível a similaridade quando comparado com o treinamento local usual do Pytorch.

In [ ]:
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)

# Parabéns!!! - Hora de se juntar a comunidade!

Parabéns por concluir esta etapa do tutorial! Se você gostou e gostaria de se juntar ao movimento em direção à proteção de privacidade, propriedade descentralizada e geração, demanda em cadeia, de dados em IA, você pode fazê-lo das seguintes maneiras!

### Dê-nos uma estrela em nosso repo do PySyft no GitHub

A maneira mais fácil de ajudar nossa comunidade é adicionando uma estrela nos nossos repositórios! Isso ajuda a aumentar a conscientização sobre essas ferramentas legais que estamos construindo.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Junte-se ao Slack!

A melhor maneira de manter-se atualizado sobre os últimos avanços é se juntar à nossa comunidade! Você pode fazer isso preenchendo o formulário em [http://slack.openmined.org](http://slack.openmined.org)

### Contribua com o projeto!

A melhor maneira de contribuir para a nossa comunidade é se tornando um contribuidor do código! A qualquer momento, você pode acessar a página de *Issues* (problemas) do PySyft no GitHub e filtrar por "Projetos". Isso mostrará todas as etiquetas (tags) na parte superior, com uma visão geral de quais projetos você pode participar! Se você não deseja ingressar em um projeto, mas gostaria de codificar um pouco, também pode procurar mais mini-projetos "independentes" pesquisando problemas no GitHub marcados como "good first issue".

- [Projetos do PySyft](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Etiquetados como Good First Issue](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Doar

Se você não tem tempo para contribuir com nossa base de códigos, mas ainda deseja nos apoiar, também pode se tornar um Apoiador em nosso Open Collective. Todas as doações vão para hospedagem na web e outras despesas da comunidade, como hackathons e meetups!

[Página do Open Collective do OpenMined](https://opencollective.com/openmined)